# Interactive seq2seq
Applying our trained encoder-decoder model so that it can produce and respond to dialogue in context. We will need to load the model, seq2seq_baseline.ckpt, which was trained on the NLP cluster with 2500 iterations and batch size of 1028.

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import warnings
import random
import json
from tensorflow.python.layers.core import Dense

In [2]:
import sys
sys.path.append("../src/models/")
sys.path.append('../src/models/agents/')
sys.path.append('../src/data/')
from agent import Agent
from parse import FBParser

### Set global variables


In [3]:
train_iterations = 3
learning_rate = 0.1
max_input_length = 6 # length of goals list
max_output_length = 20
unk_threshold = 20

### Initialize parser and agent
Using the Agent() class which subclasses the basic TfEncoderDecoder for dialogue.

In [4]:
parser = FBParser(unk_threshold=unk_threshold,
                  input_directory="../data/raw/",
                  output_directory="../data/processed/")
print("Vocab size: {}".format(parser.vocab_size))

Vocab size: 502


In [5]:
agent = Agent(vocab=parser.vocab,
              max_iter=train_iterations,
              eta=learning_rate,
              max_input_length=max_input_length,
              max_output_length=max_output_length,
              hidden_dim=64)

### Model training

In [6]:
train_data = []
with open("../data/processed/train.txt", "r") as train_file:
    for line in train_file:
        train_example = json.loads(line)
        train_data.append((
            train_example["input"],
            train_example["output"][0].split()))

In [7]:
X, y = zip(*train_data)

In [8]:
agent.fit(X, y, save_path="../models/seq2seq")

Iteration 3: loss: 46.00305223464966

Model saved in path: ../models/seq2seq


Load the saved model:

In [9]:
tf.reset_default_graph()
newAgent = Agent(vocab=parser.vocab,
              max_iter=train_iterations,
              eta=learning_rate,
              max_input_length=max_input_length,
              max_output_length=max_output_length,
              hidden_dim=64)
newAgent.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=False))

In [10]:
saver = tf.train.import_meta_graph('../models/seq2seq.meta')

In [11]:
saver.restore(newAgent.sess, '../models/seq2seq')

INFO:tensorflow:Restoring parameters from ../models/seq2seq


Print elements of the graph:

In [12]:
[n.name for n in tf.get_default_graph().as_graph_def().node]

['encoder_inputs',
 'encoder_lengths',
 'decoder_inputs',
 'decoder_targets',
 'decoder_lengths',
 'EmbedSequence/embeddings/Initializer/random_uniform/shape',
 'EmbedSequence/embeddings/Initializer/random_uniform/min',
 'EmbedSequence/embeddings/Initializer/random_uniform/max',
 'EmbedSequence/embeddings/Initializer/random_uniform/RandomUniform',
 'EmbedSequence/embeddings/Initializer/random_uniform/sub',
 'EmbedSequence/embeddings/Initializer/random_uniform/mul',
 'EmbedSequence/embeddings/Initializer/random_uniform',
 'EmbedSequence/embeddings',
 'EmbedSequence/embeddings/Assign',
 'EmbedSequence/embeddings/read',
 'EmbedSequence/embedding_lookup/axis',
 'EmbedSequence/embedding_lookup',
 'random_uniform/shape',
 'random_uniform/min',
 'random_uniform/max',
 'random_uniform/RandomUniform',
 'random_uniform/sub',
 'random_uniform/mul',
 'random_uniform',
 'Variable',
 'Variable/Assign',
 'Variable/read',
 'embedding_lookup/axis',
 'embedding_lookup',
 'encoding_layer/Rank',
 'encodin

### Test data prediction
Make sure the saved model works correctly:

In [13]:
test_data = []
with open("../data/processed/test.txt", "r") as test_file:
    for line in test_file:
        test_example = json.loads(line)
        test_data.append((
            test_example["input"],
            test_example["output"][0].split()))

Get first twenty examples:

In [14]:
X_test, y_test = zip(*test_data[:20])

In [15]:
logits = newAgent.predict(X_test)
f.write('\nPredictions:\n')
for string in newAgent.output(logits, padding=" "):
    f.write(string)
    f.write("\n")

AttributeError: 'Agent' object has no attribute 'inference_logits'